# Predicting Units Sold by Store and Product

## Business Understanding

## Data Exploration

I recieved my data from the M5 Forecasting - Accuracy competition on [Kaggle](https://www.kaggle.com/competitions/m5-forecasting-accuracy/data). It contains a 5 files: calendar.csv, sales_train_validation.csv, sample_submission.csv, sell_prices.csv, and sales_train_evaluation.csv. The calendar.csv file contains the date for every day, as well as the weekday, month, and Wal-Mart week. It also includes information of the days holiday status, and the days when snap is available. The sales_train_validation.csv contains information on the amount of units sold every day for 3019 different products across 10 different Wal-Mart stores in three different states. The sales_train_evaluation data contains the same information as the validation data, along with the correct values for the 28 day forecast. The sample_submission.csv contains an example csv for the format needed to submit the 28 day forecasts. The sell_prices.csv contains the prices at which the items were sold for every Wal-Mart week. 

In [1]:
# imports for notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [2]:
# read in the dates and their recorded unique characteristics
dates = pd.read_csv('C:/Users/TWood/Downloads/m5-forecasting-accuracy/calendar.csv', parse_dates=[0])

In [ ]:
# take a look at the dates df
dates

In [3]:
# most days have no event, replace NaN with "None"
dates.replace(np.NaN, 'None', inplace=True)

In [ ]:
len(dates['event_name_1'].unique())

In [ ]:
event_dict = dict(zip(dates['event_name_1'].unique(), range(0,31)))

In [4]:
dates.drop(columns=['weekday', 'year', 'wm_yr_wk', 'event_type_1', 'event_type_2', 'snap_TX', 'snap_WI'], inplace=True)

In [5]:
# read in the training data
val = pd.read_csv('C:/Users/TWood/Downloads/m5-forecasting-accuracy/sales_train_validation.csv')

In [ ]:
val

In [6]:
CA1_F1 = val[(val['store_id'] == 'CA_1')&(val['dept_id'] == 'FOODS_1')]

In [7]:
CA1_F1.drop(columns=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], inplace=True)

C:\Users\TWood\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [8]:
items = CA1_F1['id'].unique()

In [9]:
CA1_F1_ts = CA1_F1.melt(id_vars=['id'], var_name='d', value_name='sales')

In [10]:
CA1_F1_ts = CA1_F1_ts.merge(dates, how='inner', on='d')

In [11]:
CA1_F1_ts['lag_1'] = CA1_F1_ts['sales'].shift(periods=216*1)

In [12]:
CA1_F1_ts['lag_2'] = CA1_F1_ts['sales'].shift(periods=216*2)

In [13]:
CA1_F1_ts['lag_3'] = CA1_F1_ts['sales'].shift(periods=216*3)

In [14]:
CA1_F1_ts['lag_4'] = CA1_F1_ts['sales'].shift(periods=216*4)

In [15]:
CA1_F1_ts['lag_5'] = CA1_F1_ts['sales'].shift(periods=216*5)

In [16]:
CA1_F1_ts['lag_6'] = CA1_F1_ts['sales'].shift(periods=216*6)

In [17]:
CA1_F1_ts['lag_7'] = CA1_F1_ts['sales'].shift(periods=216*7)

In [18]:
CA1_F1_ts['lag_14'] = CA1_F1_ts['sales'].shift(periods=216*14)

In [19]:
CA1_F1_ts['lag_21'] = CA1_F1_ts['sales'].shift(periods=216*21)

In [20]:
CA1_F1_ts['lag_28'] = CA1_F1_ts['sales'].shift(periods=216*28)

In [21]:
CA1_F1_ts.set_index('date', inplace=True)

In [49]:
items[0]

'FOODS_1_001_CA_1_validation'

In [22]:
for col in ['event_name_1', 'event_name_2']:
    label = LabelEncoder().fit(CA1_F1_ts[col])
    CA1_F1_ts[col] = label.transform(CA1_F1_ts[col])

In [60]:
CA1_F1_ts

,id,d,sales,wday,month,event_name_1,event_name_2,snap_CA,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_14,lag_21,lag_28
date,,,,,,,,,,,,,,,,,,
2011-01-29,FOODS_1_001_CA_1_validation,d_1,3,1,1,19,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_1_002_CA_1_validation,d_1,0,1,1,19,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_1_003_CA_1_validation,d_1,0,1,1,19,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_1_004_CA_1_validation,d_1,0,1,1,19,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_1_005_CA_1_validation,d_1,3,1,1,19,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-24,FOODS_1_215_CA_1_validation,d_1913,0,2,4,19,3,0,0.0,0.0,0.0,0.0,4.0,6.0,2.0,0.0,4.0,0.0
2016-04-24,FOODS_1_216_CA_1_validation,d_1913,1,2,4,19,3,0,3.0,1.0,1.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0
2016-04-24,FOODS_1_217_CA_1_validation,d_1913,3,2,4,19,3,0,4.0,1.0,2.0,0.0,2.0,3.0,3.0,5.0,4.0,2.0


In [28]:
preds = []
trues = []
for item in items:
    ts = CA1_F1_ts[CA1_F1_ts['id'] == item].drop(columns=['d', 'id'])
    train = ts['2014-03-28':'2016-03-28']
    test = ts['2016-03-28':]
    X_train = train.drop('sales', axis=1)
    X_test = test.drop('sales', axis=1)
    y_train = train['sales']
    y_test = test['sales']
    lgbm = LGBMRegressor().fit(X_train, y_train)
    forecast = []
    for X in X_test.index:
        pred = lgbm.predict(X_test[[X]])
        forecast.append(pred)
        X_test[X]['sales'] = pred
    preds.append(forecast)
    trues.append(y_test)

DatetimeIndex(['2016-03-28', '2016-03-29', '2016-03-30', '2016-03-31',
               '2016-04-01', '2016-04-02', '2016-04-03', '2016-04-04',
               '2016-04-05', '2016-04-06', '2016-04-07', '2016-04-08',
               '2016-04-09', '2016-04-10', '2016-04-11', '2016-04-12',
               '2016-04-13', '2016-04-14', '2016-04-15', '2016-04-16',
               '2016-04-17', '2016-04-18', '2016-04-19', '2016-04-20',
               '2016-04-21', '2016-04-22', '2016-04-23', '2016-04-24'],
              dtype='datetime64[ns]', name='date', freq=None)


ValueError: Expected 2D array, got scalar array instead:
array=wday.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [67]:
preds

[array([ 1.00415634,  0.36938658,  0.18359893,  1.82603926,  0.92233571,
         1.18994156,  0.58138525,  1.00061306, -0.03432485,  0.39363485,
         0.58870642,  0.46138554,  1.15491264,  1.00418386,  0.08148938,
         0.36083233,  1.16409658,  1.21367663, -0.08313242,  1.57946718,
         1.36395335, -0.03010396,  1.80428693,  0.23024728,  0.64792587,
        -0.0587915 ,  0.77697793,  0.73500074]),
 array([ 0.17471345,  0.57953627,  0.61683382,  0.62935702, -0.25091502,
         1.13729578,  0.70864557,  0.67411906,  0.2586207 ,  0.09640887,
         0.25854004,  0.0333561 ,  1.0912429 ,  0.93229044,  0.34669764,
         0.17073497,  0.58404143,  0.27477292, -0.04704554,  0.8578524 ,
         0.7238624 ,  0.06767575,  0.17918604, -0.08508529,  0.09632065,
         0.36846176,  0.21175911,  0.66937566]),
 array([ 2.68356576,  0.52372729, -0.14735986,  0.7765607 ,  0.67632774,
         1.17753789,  1.88924428,  2.19625143, -0.09045782,  0.4676692 ,
         1.47875791,  0.79

In [70]:
trues

[date
 2016-03-28    2
 2016-03-29    1
 2016-03-30    1
 2016-03-31    0
 2016-04-01    4
 2016-04-02    0
 2016-04-03    0
 2016-04-04    4
 2016-04-05    1
 2016-04-06    3
 2016-04-07    0
 2016-04-08    1
 2016-04-09    0
 2016-04-10    2
 2016-04-11    2
 2016-04-12    0
 2016-04-13    1
 2016-04-14    1
 2016-04-15    0
 2016-04-16    2
 2016-04-17    0
 2016-04-18    4
 2016-04-19    1
 2016-04-20    1
 2016-04-21    0
 2016-04-22    1
 2016-04-23    1
 2016-04-24    0
 Name: sales, dtype: int64,
 date
 2016-03-28    0
 2016-03-29    1
 2016-03-30    0
 2016-03-31    0
 2016-04-01    0
 2016-04-02    0
 2016-04-03    0
 2016-04-04    0
 2016-04-05    0
 2016-04-06    1
 2016-04-07    0
 2016-04-08    0
 2016-04-09    0
 2016-04-10    0
 2016-04-11    1
 2016-04-12    0
 2016-04-13    0
 2016-04-14    1
 2016-04-15    1
 2016-04-16    3
 2016-04-17    1
 2016-04-18    0
 2016-04-19    0
 2016-04-20    1
 2016-04-21    2
 2016-04-22    0
 2016-04-23    0
 2016-04-24    0
 Name: s

In [77]:
RMSEs = []
for i in range(216):
    RMSEs.append(mean_squared_error(trues[i], preds[i], squared=False))

In [80]:
np.mean(RMSEs)

1.554662679509895

In [68]:
CA1_F1_ts

,id,d,sales,wday,month,event_name_1,event_name_2,snap_CA,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_14,lag_21,lag_28
date,,,,,,,,,,,,,,,,,,
2011-01-29,FOODS_1_001_CA_1_validation,d_1,3,1,1,19,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_1_002_CA_1_validation,d_1,0,1,1,19,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_1_003_CA_1_validation,d_1,0,1,1,19,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_1_004_CA_1_validation,d_1,0,1,1,19,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_1_005_CA_1_validation,d_1,3,1,1,19,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-24,FOODS_1_215_CA_1_validation,d_1913,0,2,4,19,3,0,0.0,0.0,0.0,0.0,4.0,6.0,2.0,0.0,4.0,0.0
2016-04-24,FOODS_1_216_CA_1_validation,d_1913,1,2,4,19,3,0,3.0,1.0,1.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0
2016-04-24,FOODS_1_217_CA_1_validation,d_1913,3,2,4,19,3,0,4.0,1.0,2.0,0.0,2.0,3.0,3.0,5.0,4.0,2.0


In [ ]:
trial.drop(columns=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], inplace=True)

In [ ]:
trial_ts = trial.melt(var_name='d', value_name='sales')

In [ ]:
trial_ts = trial_ts.merge(dates, how='inner', on='d')

In [ ]:
trial_ts.set_index('date', inplace=True)

In [ ]:
trial_ts['lag_1'] = trial_ts['sales'].shift(periods=216*1)

In [ ]:
trial_ts['lag_2'] = trial_ts['sales'].shift(periods=216*2)

In [ ]:
trial_ts['lag_3'] = trial_ts['sales'].shift(periods=216*3)

In [ ]:
trial_ts['lag_4'] = trial_ts['sales'].shift(periods=216*4)

In [ ]:
trial_ts['lag_5'] = trial_ts['sales'].shift(periods=216*5)

In [ ]:
trial_ts['lag_6'] = trial_ts['sales'].shift(periods=216*6)

In [ ]:
trial_ts['lag_7'] = trial_ts['sales'].shift(periods=216*7)

In [ ]:
trial_ts['lag_14'] = trial_ts['sales'].shift(periods=216*14)

In [ ]:
trial_ts['lag_21'] = trial_ts['sales'].shift(periods=216*21)

In [ ]:
trial_ts['lag_28'] = trial_ts['sales'].shift(periods=216*28)

In [ ]:
train = trial_ts['2014-03-28':'2016-03-28']

In [ ]:
test = trial_ts['2016-03-28':]

In [ ]:
X_train = train.drop(['sales', 'd', 'wm_yr_wk', 'event_name_1', 'event_name_2'], axis=1)
X_test = test.drop(['sales', 'd', 'wm_yr_wk', 'event_name_1', 'event_name_2'], axis=1)
y_train = train['sales']
y_test = test['sales']

In [ ]:
y_test

In [ ]:
label = LabelEncoder()
X_train['id'] = label.fit_transform(X_train['id'])
X_test['id'] = label.transform(X_test['id'])

In [ ]:
trial_ts

In [ ]:
X_train

In [ ]:
rf2 = RandomForestRegressor(max_depth=20)
rf2.fit(X_train, y_train)

In [ ]:
rf2.score(X_train, y_train)

In [ ]:
rf2.score(X_test, y_test)

In [ ]:
rf2_preds = rf2.predict(X_test)

In [ ]:
mean_squared_error(y_test, rf2_preds)

In [ ]:
val['dept_id'].unique()

In [ ]:
CA1_f1 = val[(val['store_id'] == 'CA_1')&(val['dept_id'] == 'FOODS_1')]

In [ ]:
CA1_f1_melt = CA1_f1.drop(columns=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])

In [ ]:
CA1_f1_ts = CA1_f1_melt.melt(id_vars=['id'], var_name='d', value_name='sales')

In [ ]:
CA1_f1_ts

In [ ]:
CA1_f1_ts = CA1_f1_ts.merge(dates, how='inner', on='d')

In [ ]:
CA1_f1_ts

In [ ]:
CA1_f1_ts.loc['2011-01-30']

In [ ]:
CA1_f1_ts.set_index('date', inplace=True)

In [ ]:
CA1_f1_ts['lag_1'] = CA1_f1_ts['sales'].shift(periods=216)

In [ ]:
CA1_f1_ts['lag_2'] = CA1_f1_ts['sales'].shift(periods=(216*2))

In [ ]:
CA1_f1_ts['lag_3'] = CA1_f1_ts['sales'].shift(periods=(216*3))

In [ ]:
CA1_f1_ts['lag_4'] = CA1_f1_ts['sales'].shift(periods=(216*4))

In [ ]:
CA1_f1_ts['lag_5'] = CA1_f1_ts['sales'].shift(periods=(216*5))

In [ ]:
CA1_f1_ts['lag_6'] = CA1_f1_ts['sales'].shift(periods=(216*6))

In [ ]:
CA1_f1_ts['lag_7'] = CA1_f1_ts['sales'].shift(periods=(216*7))

In [ ]:
CA1_f1_ts['lag_14'] = CA1_f1_ts['sales'].shift(periods=(216*14))

In [ ]:
CA1_f1_ts['lag_21'] = CA1_f1_ts['sales'].shift(periods=(216*21))

In [ ]:
CA1_f1_ts['lag_28'] = CA1_f1_ts['sales'].shift(periods=(216*28))

In [ ]:
CA1_f1_ts

In [ ]:
train = CA1_f1_ts['2014-03-28':'2016-03-28']

In [ ]:
test = CA1_f1_ts['2016-03-28':]

In [ ]:
# item 7, 100, and 165 are missing, thus 216 per day
test['id'].unique()

In [ ]:
train

In [ ]:
test

In [ ]:
X_train = train.drop(['sales', 'd', 'wm_yr_wk', 'event_name_1', 'event_name_2'], axis=1)
X_test = test.drop(['sales', 'd', 'wm_yr_wk', 'event_name_1', 'event_name_2'], axis=1)
y_train = train['sales']
y_test = test['sales']

In [ ]:
y_test

In [ ]:
label = LabelEncoder()
X_train['id'] = label.fit_transform(X_train['id'])
X_test['id'] = label.transform(X_test['id'])

In [ ]:
rf1 = RandomForestRegressor(max_depth=20)
rf1.fit(X_train, y_train)

In [ ]:
rf1.score(X_train, y_train)

In [ ]:
rf1.score(X_test, y_test)

In [ ]:
rf1_preds = rf1.predict(X_test)

In [ ]:
rf1_preds = pd.Series(rf1_preds).set_axis(y_test.index)

In [ ]:
fig, ax = plt.subplots()
ax.plot(rf1_preds)
ax.plot(y_test)

In [ ]:
mean_squared_error(rf1_preds, y_test, squared=False)

In [ ]:
lgbm_1 = LGBMRegressor()
lgbm_1.fit(X_train, y_train)

In [ ]:
lgbm_1.score(X_train, y_train)

In [ ]:
lgbm_1.score(X_test, y_test)

In [ ]:
xgb_1 = XGBRegressor()
xgb_1.fit(X_train, y_train)

In [ ]:
xgb_1.score(X_train, y_train)

In [ ]:
xgb_1.score(X_test, y_test)

In [ ]:
CA1_join = CA_1[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']]

In [ ]:
#to_melt = train.drop(columns=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])

In [ ]:
#train_ts = to_melt.melt(id_vars=['id'], var_name='d', value_name='sales')

In [ ]:
#to_join = train[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']]

In [ ]:
train_ts = train_ts.merge(to_join, on='id', how='inner')

In [ ]:
train_ts.to_csv('./timeseries.csv')

In [ ]:
train_ts.merge(dates, on='d', how='inner')

In [ ]:
train_ts['value'].max()

In [ ]:
train_ts = pd.melt(train, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='day')

In [ ]:
train_ts

In [ ]:
train = pd.read_csv('C:/Users/TWood/Downloads/m5-forecasting-accuracy/sales_train_evaluation.csv')

In [ ]:
val_ts = pd.melt(val.head(), id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d')

In [ ]:
val_ts = val_ts.merge(dates, on='d')

In [ ]:
val_ts.set_index('date')

In [ ]:
df.describe()

In [ ]:
pd.read_csv('C:/Users/TWood/Downloads/m5-forecasting-accuracy/sample_submission.csv')

In [ ]:
pd.read_csv('C:/Users/TWood/Downloads/m5-forecasting-accuracy/sell_prices.csv')